#🟦 **Cohort Analysis - Data Setup (bigquery)**

## 🔵1. **Data Ingestion: Local to Cloud Storage (GCS)**

In [9]:
# 1. CSVs from Notebook → Cloud Storage

from google.cloud import storage

# Define your GCS bucket
bucket_name = "cohort_analysis_raw_data"

# Local files (already in Notebook root)
local_files = {
    "/ecom_orders.csv": "raw/ecom_orders.csv"
}

# Initialize GCS client
client = storage.Client()              # Connection to GCS
bucket = client.bucket(bucket_name)    # identifiy bucket_name

# Loop through all local → cloud file pairs
for local_path, gcs_path in local_files.items():

    # Create the Placeholder
    blob = bucket.blob(gcs_path)

    # Upload the local file into this blob (cloud file)
    blob.upload_from_filename(local_path)

    # Print confirmation: local → cloud
    print(f"Uploaded {local_path} → gs://{bucket_name}/{gcs_path}")



Uploaded /ecom_orders.csv → gs://cohort_analysis_raw_data/raw/ecom_orders.csv


## 🔵 **2. Creating the Cohort Analysis Dataset (Schema)**

In [10]:
%%bigquery
CREATE SCHEMA `hybrid-sentry-479215-n9.cohort_analysis`
OPTIONS (
    description = 'Dataset for Cohort_Analysis'
);

Executing query with job ID: fff0eeab-65a9-4e10-95b2-0b2543fc8445
Query executing: 0.39s


ERROR:
 409 Already Exists: Dataset hybrid-sentry-479215-n9:cohort_analysis; reason: duplicate, message: Already Exists: Dataset hybrid-sentry-479215-n9:cohort_analysis

Location: europe-west3
Job ID: fff0eeab-65a9-4e10-95b2-0b2543fc8445



## 🔵 **3. Defining and Creating the Target Table**

In [11]:
%%bigquery

CREATE OR REPLACE TABLE `hybrid-sentry-479215-n9.cohort_analysis.ecom_orders` (

row_id INT,
customer_id STRING,
order_date DATE,
order_id STRING,
sales FLOAT64

)

OPTIONS(description = "raw transactional data");

Query is running:   0%|          |

""


## 🔵 **3. Loading Data into the Destination Table (Load)**

In [12]:
%%bigquery

LOAD DATA INTO
  `hybrid-sentry-479215-n9.cohort_analysis.ecom_orders`
FROM FILES (
  format = "CSV",
  uris = ["gs://cohort_analysis_raw_data/raw/ecom_orders.csv"],
  skip_leading_rows = 1,
  field_delimiter = ","
);

Query is running:   0%|          |

""


## 🔵 **Test Query**

In [13]:
%%bigquery

SELECT *
FROM  `hybrid-sentry-479215-n9.cohort_analysis.ecom_orders`;

Query is running:   0%|          |

Downloading:   0%|          |

,row_id,customer_id,order_date,order_id,sales
0,1,CUST001,2024-06-02,ORD1000,76.93
1,2,CUST001,2024-07-26,ORD1001,251.35
2,3,CUST001,2024-10-12,ORD1002,135.91
3,797,CUST001,2024-11-15,ORD1796,15.89
4,973,CUST001,2024-04-12,ORD1972,295.76
...,...,...,...,...,...
995,815,CUST199,2024-09-30,ORD1814,283.50
996,550,CUST200,2024-02-14,ORD1549,41.99
997,551,CUST200,2024-03-02,ORD1550,126.87
998,552,CUST200,2024-04-27,ORD1551,159.89
